In [ ]:
import pandas as pd

In [3]:
data = pd.read_csv("Robust_scaller_result.csv",encoding="cp949")

In [4]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [5]:
act = act-1
act = -act

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 39) (11736, 39) (27381,) (11736,)


In [7]:
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()

In [8]:
import xgboost as xgb

from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [9]:
result_list=[]

In [14]:
params = {
    'booster' : ['gbtree'],
    'eta': [0.15],
    'gamma':[0],
    'max_depth': [30,35,40,45,50],
    'min_child_weight':[1,2,3],
    'random_state':[99],
    'n_estimators':[300,400,500]
}
xgbs = xgb.XGBClassifier(use_label_encoder=False,eval_metric='logloss',)

from sklearn.model_selection import GridSearchCV


In [15]:
for a in keys_list:
    xgb_gird = GridSearchCV(xgbs,param_grid=params, scoring="accuracy",n_jobs=3,verbose=1)

    xgb_gird.fit(data_sets[a][0], data_sets[a][1])

    pred_y = xgb_gird.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    result_list.append((a,xgb_gird.best_params_,pr,r,f1,rca,cfm))
    
    



Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  8.4min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 37.1min
[Parallel(n_jobs=3)]: Done 225 out of 225 | elapsed: 43.0min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 25.3min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 109.8min
[Parallel(n_jobs=3)]: Done 225 out of 225 | elapsed: 126.8min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 23.6min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 103.9min
[Parallel(n_jobs=3)]: Done 225 out of 225 | elapsed: 120.3min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  9.9min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 44.1min
[Parallel(n_jobs=3)]: Done 225 out of 225 | elapsed: 50.8min finished


Fitting 5 folds for each of 45 candidates, totalling 225 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 22.7min
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 99.6min
[Parallel(n_jobs=3)]: Done 225 out of 225 | elapsed: 115.2min finished


In [16]:
result_list

[('enn',
  {'booster': 'gbtree',
   'eta': 0.15,
   'gamma': 0,
   'max_depth': 30,
   'min_child_weight': 1,
   'random_state': 99},
  0.9350226244343891,
  0.9686855428464279,
  0.9515564560692575,
  0.6488287527316252,
  array([[  352,   718],
         [  334, 10332]], dtype=int64)),
 ('ada',
  {'booster': 'gbtree',
   'eta': 0.15,
   'gamma': 0,
   'max_depth': 35,
   'min_child_weight': 1,
   'random_state': 99},
  0.9302078726227333,
  0.985936621038815,
  0.9572618451595286,
  0.6242767217343608,
  array([[  281,   789],
         [  150, 10516]], dtype=int64)),
 ('smte',
  {'booster': 'gbtree',
   'eta': 0.15,
   'gamma': 0,
   'max_depth': 30,
   'min_child_weight': 1,
   'random_state': 99},
  0.9408301158301158,
  0.9138383648978061,
  0.927137829354133,
  0.670470584318062,
  array([[ 457,  613],
         [ 919, 9747]], dtype=int64)),
 ('smtt',
  {'booster': 'gbtree',
   'eta': 0.15,
   'gamma': 0,
   'max_depth': 35,
   'min_child_weight': 1,
   'random_state': 99},
  0.930